In [2]:
#导入包(PypI)
import os
import pymongo
import requests
from lxml import etree
import pandas as pd
import time
import random
import asyncio
import json

In [7]:
#连接MongoDB
myclient=pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient["Steam平台用户活跃数据"]#进行操作的库名
mycol=mydb["MostPlayedGamesTop500"]#进行操作的集合名

In [8]:
data=[]
for doc in mycol.find():
    data.append(doc)
df=pd.DataFrame(data)
df.head()

,_id,ID,name(Chinese),top_players
0,64560396ed21cfd4942fa0b5,578080,绝地求生,3236027
1,64560396ed21cfd4942fa0b6,730,反恐精英：全球攻势,1533939
2,64560396ed21cfd4942fa0b7,570,刀塔2,1291328
3,64560396ed21cfd4942fa0b8,1599340,Lost Ark,1174722
4,64560396ed21cfd4942fa0b9,1091500,赛博朋克 2077,1041355


In [9]:
link=[]
for i in range(len(df['ID'])):
    link.append("https://store.steampowered.com/app/"+str(df['ID'][i]))
len(link)

500

In [10]:
game_info=[]
wait_time=random.uniform(3,5)#生成随机等待时间
for l in link:
    try:
        response=requests.get(l)
        response.raise_for_status()#检查是否有错误的响应码
        html=etree.HTML(response.content)
        name=''
        try:
            name=html.xpath('//*[@id="genresAndManufacturer"]/text()[normalize-space()]')[0]
        except:
            pass
        tags=[]
        try:
            tags=html.xpath('//*[@id="genresAndManufacturer"]/span/a/text()')
            tags=', '.join(tags)
        except:
            pass
        languages=[]
        try:
            languages=html.xpath('//table[@class="game_language_options"]//td[1]/text()')
            languages=[lang.strip() for lang in languages]#使用strip()方法去除每个元素中的换行符、制表符和空格
            languages=', '.join(languages)
        except:
            pass
        developer=''
        try:
            developer=html.xpath('//*[@id="genresAndManufacturer"]/div[1]/a/text()')
        except:
            pass
        publisher=''
        try:
            publisher=html.xpath('//*[@id="genresAndManufacturer"]/div[2]/a/text()')
        except:
            pass
        release_date=''
        try:
            release_date=html.xpath('//*[@id="game_highlights"]/div[1]/div/div[3]/div[2]/div[2]/text()')
        except:
            pass
        game_info.append({
            'name(English)': name,
            'tags': tags,
            'languages': languages,
            'developer': developer,
            'publisher': publisher,
            'release_date': release_date
        })
        time.sleep(wait_time)#请求延迟时间
    except requests.exceptions.HTTPError as errh:
        print("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print("Something went wrong:",err)

game_df=pd.DataFrame(game_info)
game_df=game_df.fillna('')#将空值填充为''
print(len(game_df))

500


In [1]:
game_df=pd.DataFrame(game_info)
game_df=game_df.fillna('')#将空值填充为''
game_df['ID']=df['ID']
game_df['name(Chinese)']=df['name(Chinese)']
game_df['top_players']=df['top_players']
game_df['developer']=game_df['developer'].astype(str).str.replace("['", "").str.replace("']", "").str.replace("[]", "")
game_df['publisher']=game_df['publisher'].astype(str).str.replace("['", "").str.replace("']", "").str.replace("[]", "")
game_df['release_date']=game_df['release_date'].astype(str).str.replace("['", "").str.replace("']", "").str.replace("[]", "")
game_df=game_df.reindex(columns=['ID','name(English)','name(Chinese)','tags','languages','top_players','developer','publisher','release_date'])
print(len(game_df))
game_df.head()

NameError: name 'pd' is not defined

In [12]:
#更新MongoDB的Steam平台用户活跃数据.MostPlayedGamesTop500DetailedInformation
mycol=mydb["MostPlayedGamesTop500DetailedInformation"]
game_dict=game_df.to_dict('records')
mycol.insert_many(game_dict)